In [1]:
import os

from datetime import timezone, datetime

from crypto_trades_downloader.timescaledb_util import TimeScaleDBUtil
import pandas as pd

pd.options.display.precision = 10

# PostgreSQL設定
pg_config = {
    'user': os.environ['POSTGRES_USER'],
    'password': os.environ['POSTGRES_PASSWORD'],
    'host': os.environ['POSTGRES_HOST'],
    'port': os.environ['POSTGRES_PORT'],
    'database': os.environ['POSTGRES_DATABASE']
}

dbutil = TimeScaleDBUtil(user = pg_config['user'], password = pg_config['password'], host = pg_config['host'], port = pg_config['port'], database = pg_config['database'])

In [7]:
ex_a_name = 'ftx'
ex_b_name = 'binance'
target_pair = "XRP/BTC"
from_date = '2022-01-01 00:00:00.000+00'
to_date = '2022-01-05 23:59:59.999+00'
target_diff = 0.15

table_name = dbutil.get_trade_table_name(ex_a_name, target_pair)
df_trades_a = dbutil.read_sql_query(sql = f'SELECT * FROM "{table_name}" WHERE datetime BETWEEN \'{from_date}\' AND \'{to_date}\' ORDER BY datetime')
df_trades_a = df_trades_a.rename(columns={'price': 'price_a'})
df_trades_a['exchange'] = ex_a_name

table_name = dbutil.get_trade_table_name(ex_b_name, target_pair)
df_trades_b = dbutil.read_sql_query(sql = f'SELECT * FROM "{table_name}" WHERE datetime BETWEEN \'{from_date}\' AND \'{to_date}\' ORDER BY datetime')
df_trades_b = df_trades_b.rename(columns={'price': 'price_b'})
df_trades_b['exchange'] = ex_b_name

In [8]:
df_ab_trades_nona = pd.concat([df_trades_a, df_trades_b]).sort_values('datetime', ascending=True).fillna(method='ffill').dropna().reset_index(drop=True)
df_ab_trades_nona['price_diff'] = df_ab_trades_nona['price_a'] - df_ab_trades_nona['price_b']
df_ab_trades_nona['price_diff_pct'] = df_ab_trades_nona['price_diff'] / df_ab_trades_nona['price_a'] * 100
df_ab_trades_nona['profitable'] = False
df_ab_trades_nona.loc[(df_ab_trades_nona['price_diff_pct'] >= target_diff) | (df_ab_trades_nona['price_diff_pct'] <= -target_diff), 'profitable'] = True
df_ab_trades_nona['inverse_profitable'] = ~df_ab_trades_nona['profitable']
df_ab_trades_nona['profitable_label'] = df_ab_trades_nona['inverse_profitable'].cumsum()

In [9]:
df_ab_trades_nona_timediff = df_ab_trades_nona.sort_values('datetime', ascending=False)
df_ab_trades_nona_timediff['datetime_diff'] = -df_ab_trades_nona_timediff['datetime'].diff().dt.total_seconds()
df_ab_trades_nona_timediff.dropna(inplace=True)
df_ab_trades_nona_timediff = df_ab_trades_nona_timediff.sort_values('datetime', ascending=True).reset_index(drop=True)

In [10]:
df_ab_trades_nona_timediff_profitable = df_ab_trades_nona_timediff[df_ab_trades_nona_timediff['profitable'] == True]
df_ab_trades_nona_timediff_profitable_group = df_ab_trades_nona_timediff_profitable.groupby("profitable_label")

In [11]:
from scipy import stats
import numpy as np

print(f"{ex_a_name} vs {ex_b_name}: {target_pair}")
print(f"{from_date} から {to_date}の集計結果です\n")

print(f"サンプル数は{len(df_ab_trades_nona_timediff)}でした")
print(f"価格乖離が{target_diff}%以上のサンプル数は{len(df_ab_trades_nona_timediff_profitable)}でした\n")
print(f"価格乖離が{target_diff}%以上の時間は{df_ab_trades_nona_timediff_profitable['datetime_diff'].sum() / 60 / 60:.02f}時間でした\n")

print(f"\n価格乖離の分布")
for x in range(0, 101, 1):
    percentile = stats.scoreatpercentile(df_ab_trades_nona['price_diff_pct'], x)
    print(f"{x} percentile: {percentile:.3f}%")

df_time_diff = df_ab_trades_nona_timediff_profitable_group['datetime_diff'].sum()

print(f"\n価格乖離の絶対値が{target_diff}%を超えた連続時間の分布")
for x in range(0, 101, 1):
    percentile = stats.scoreatpercentile(df_time_diff, x)
    print(f"{x} percentile: {percentile:.2f} sec")

ftx vs binance: XRP/BTC
2022-01-01 00:00:00.000+00 から 2022-01-05 23:59:59.999+00の集計結果です

サンプル数は87497でした
価格乖離が0.15%以上のサンプル数は5186でした

価格乖離が0.15%以上の時間は4.68時間でした


価格乖離の分布
0 percentile: -1.881%
1 percentile: -0.169%
2 percentile: -0.168%
3 percentile: -0.166%
4 percentile: -0.113%
5 percentile: -0.112%
6 percentile: -0.112%
7 percentile: -0.112%
8 percentile: -0.112%
9 percentile: -0.112%
10 percentile: -0.111%
11 percentile: -0.111%
12 percentile: -0.110%
13 percentile: -0.056%
14 percentile: -0.056%
15 percentile: -0.056%
16 percentile: -0.056%
17 percentile: -0.056%
18 percentile: -0.056%
19 percentile: -0.056%
20 percentile: -0.056%
21 percentile: -0.056%
22 percentile: -0.056%
23 percentile: -0.056%
24 percentile: -0.056%
25 percentile: -0.056%
26 percentile: -0.056%
27 percentile: -0.056%
28 percentile: -0.056%
29 percentile: -0.056%
30 percentile: -0.056%
31 percentile: -0.056%
32 percentile: -0.056%
33 percentile: -0.055%
34 percentile: -0.055%
35 percentile: -0.055%
36 percentile: